In [1]:
"""!pip install Flask==1.1.2
!pip install matplotlib==3.3.4
!pip install numpy==1.20.1
!pip install pandas==1.3.5
!pip install scikit_learn==0.24.1
!pip install seaborn==0.11.1"""

'!pip install Flask==1.1.2\n!pip install matplotlib==3.3.4\n!pip install numpy==1.20.1\n!pip install pandas==1.3.5\n!pip install scikit_learn==0.24.1\n!pip install seaborn==0.11.1'

In [2]:
import pandas as pd
import pickle
import numpy as np

In [3]:
from zipfile import ZipFile
import urllib.request
from io import BytesIO

In [4]:
folder = urllib.request.urlopen('https://s3.amazonaws.com/hackerday.datascience/50/dataset.zip')
zipfile = ZipFile(BytesIO(folder.read()))
zipfile.namelist()

['test.csv',
 '__MACOSX/',
 '__MACOSX/._test.csv',
 'train.csv',
 '__MACOSX/._train.csv',
 'test_data_subset.csv',
 '__MACOSX/._test_data_subset.csv']

In [5]:
test_data = pd.read_csv(zipfile.open("test_data_subset.csv"))    

In [9]:
#load the columns to drop file
columns_to_drop=pd.read_csv("columns_to_drop.csv")
columns_to_Retain = list(set(test_data.columns.values) - set(columns_to_drop.colnames.values))
test_data = test_data[columns_to_Retain]

column_datatypes = test_data.dtypes
categorical_columns = list(column_datatypes[column_datatypes=="object"].index.values)

In [11]:
# 1. Ensure `test_data` is an explicit copy before modification:
test_data = test_data.copy()

# 2. Then loop and use .loc for safe assignment:
for cf1 in categorical_columns:
    filename = cf1 + ".sav"
    le = pickle.load(open(filename, 'rb'))
    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    
    test_data.loc[:, cf1] = test_data[cf1].map(lambda x: le_dict.get(x, -1))


In [12]:
test_data_id = test_data['id']
test_data = test_data.drop('id',axis=1)

In [13]:
Column_datatypes= test_data.dtypes
Integer_columns = list(Column_datatypes.where(lambda x: x =="int64").dropna().index.values)
#convert the int64 columns categorical
test_data[Integer_columns] = test_data[Integer_columns].astype('category',copy=False)

In [14]:
#load the saved model and predict on the test data
tunedmodel_rf = pickle.load(open("tunedmodel_rf", 'rb'))

In [17]:
train_cols = tunedmodel_rf.best_estimator_.feature_names_in_

# Align order and columns
test_data = test_data.copy()
test_data = test_data.reindex(columns=train_cols, fill_value=0)

# Confirm alignment
assert list(test_data.columns) == list(train_cols)

# Now safe to predict
y_test_predict = tunedmodel_rf.predict(test_data)
test_data['predict_loss'] = y_test_predict
test_data['id'] = test_data_id
test_data = test_data[['id', 'predict_loss']]


In [18]:
test_data

,id,predict_loss
0,4,7.213629
1,6,7.492909
2,9,7.268176
3,12,7.206299
4,15,7.215372
